In [10]:
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-0125")

class SearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="""
        이 쿼리는 쿼리에 해당하는 정보를 웹사이트에서 찾아야합니다.
        예) 쿼리: Research about the XZ backdoor
        """
    )

class SearchToolWithDuck(BaseTool):
    name = "SearchToolWithDuck"
    description = "DuckDuckGo API를 사용하여 검색 결과를 가져옵니다."
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)

class SearchToolWithWiKi(BaseTool):
    name = "SearchToolWithWiKi"
    description = "Wikipedia API를 사용하여 검색 결과를 가져옵니다."
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, query):
        wiki = WikipediaAPIWrapper()
        return wiki.run(query)

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        SearchToolWithWiKi(),
        SearchToolWithDuck(),
    ],
)

prompt_base = "If wiki doesn't have any information, give it to me at DuckDuckGo."

prompt = "Research about the XZ backdoor"

output = agent.invoke(prompt_base+prompt)

with open("output.txt", "w", encoding="utf-8") as file:
    file.write(str(output))



> Entering new AgentExecutor chain...

Invoking: `SearchToolWithWiKi` with `{'query': 'Research about the XZ backdoor'}`


Page: XZ Utils backdoor
Summary: On 29 March 2024, software developer Andres Freund reported that he had found a maliciously introduced backdoor in the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 released in February 2024.While xz is commonly present in most Linux distributions, the backdoor only targeted Debian- and RPM-based systems running on the x86-64 architecture. At the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions.The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux systems. The issue has been assigned a CVSS score of 10.0, the highest possible score.



Page: Supply chain attack
Summary: A supply chain attack is a cyber-attack that seeks 